In [28]:
import  requests as re
from bs4 import BeautifulSoup
from pandas import DataFrame, Series
import numpy as np
import pandas as pd
import csv

def crawling(code):    #크롤링 함수
    url = re.get('http://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A%s'%code)
    url = url.content
    html = BeautifulSoup(url,'html.parser')
    body = html.find('body')
    fn_body = body.find('div',{'class':'fng_body asp_body'})
    ur_table = fn_body.find('div',{'id': 'div15'})
    table = ur_table.find('div',{'id':'highlight_D_Y'})
    tbody = table.find('tbody')
    tr = tbody.find_all('tr')
    Table = DataFrame()            #테이블 구성

    price = body.find('span',{'id':'svdMainChartTxt11'}) #현재 주가 크롤링
    price = price.text
    price = price.replace(',','')
    n_price = float(price)
        
    around_PER = body.find('div',{'id': 'corp_group2'}).find_all('dd')  #PER 크롤링
    dd_list = []
    for tag in around_PER:
        dd_list.append(tag.text)
    around_PER = dd_list[5]

    for i in tr:        #테이블 항목 가져오기
        category = i.find('span',{'class':'txt_acd'})
        if(category==None):
            category=i.find('th')
        category = category.text.strip()
        value_list = []
        j = i.find_all('td',{'class','r'})
        
        for value in j:
            temp = value.text.replace(',','').strip()
            try:
                temp = float(temp)
                value_list.append(temp)
            except:
                value_list.append(0)
        Table['%s'%(category)] = value_list

            
        thead = table.find('thead')               #기간 가져오기
        tr_2 = thead.find('tr',{'class','td_gapcolor2'}).find_all('th')
        year_list = []
        for i in tr_2:
            try:
                temp_year = i.find('span', {'class','txt_acd'}).text
            except:
                temp_year = i.text
                
            year_list.append(temp_year)
        Table.index = year_list
    return Table

def new_table(Table):    #기존의 데이터로 예측한 미래 5년의 데이터
    new_t = Table.T.iloc[[16,17,18,19,21]]    #ROA, ROE, EPS, BPS, PER만 추출
    ROA_avg = new_t.mean(axis=1)[0]
    ROE_avg = new_t.mean(axis=1)[1]
    bf_num = -2
    new_column = []
    for i in range(0,6):
        new_year = 2023+i
        new_t['%d'%new_year] = None
        for j in range(2,6):
            if(new_t.loc['BPS'][-j] != 0):
                break
            else:
                bf_num = bf_num-1
        new_EPS = new_t.loc['BPS'][bf_num]*(ROE_avg/100)
        new_BPS = new_t.loc['BPS'][bf_num] + new_EPS
        bf_num = -2
            
        new_t['%d'%new_year] = ROA_avg,ROE_avg,new_EPS, new_BPS, 'NAN'
    return new_t

def calculate_price(new_t):    #마지막 예측년도(2028년)의 BPS를 현재가치로 변환
    new_BPS = new_t.loc['BPS'][-1]
    new_price = new_BPS/((1.15)**8)
    print("업종 PER : "+around_PER)  
    print("현재 주가 :" + price)
    print("현재가치 : " ,round(new_price,2))
    display(new_t)
    print("투자 자격 : ", end="")
    print(("미달", "적합")[1.5*n_price<new_price])    

code= input("주식 종목코드: ")
data_table = crawling(code)
newt = new_table(data_table)
calculate_price(newt)


주식 종목코드: 005930
업종 PER : 21.39
현재 주가 :55000
현재가치 :  32648.01


,2015/12,2016/12,2017/12,2018/12,2019/12,2020/12(E),2021/12(E),2022/12(E),2023,2024,2025,2026,2027,2028
ROA,8.07,9.01,14.96,13.83,6.28,6.94,8.96,9.12,9.64625,9.64625,9.64625,9.64625,9.64625,9.64625
ROE,11.16,12.48,21.01,19.63,8.69,9.46,12.09,12.28,13.35,13.35,13.35,13.35,13.35,13.35
EPS,2198.00,2735.00,5421.00,6024.00,3166.00,3658.00,5022.00,5547.00,6286.25,7125.46,8076.71,9154.95,10377.1,11762.5
BPS,21903.00,24340.00,28971.00,35342.00,37528.00,39853.00,43256.00,47088.00,53374.2,60499.7,68576.4,77731.4,88108.5,99871
PER,11.47,13.18,9.40,6.42,17.63,15.03,10.95,9.91,NAN,NAN,NAN,NAN,NAN,NAN


투자 자격 : 미달


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:91: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
